In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

In [2]:
def url(isin):
    return("https://www.justetf.com/de/etf-profile.html?query={0}&groupField=index&from=search&isin={0}#overview"\
          .format(isin))

def get_price_stock(soup_base):
    assert isinstance(soup_base, type(BeautifulSoup())), "Input is no BeautifulSoup object!"
    price_dict = {}
    
    price_obj = soup_base.find_all("div", {"class": "infobox"})[0].find_all("div", {"class": "val"})[0].find_all("span")
    currency = price_obj[0].text
    price = float(price_obj[1].text.replace(".", "").replace(",", "."))
    
    price_dict["Currency"] = currency
    price_dict["Price"] = price
    price_dict["Datum"] = date.today().strftime("%d.%m.%Y")

    return(price_dict)

def get_prices(isin_list):
    stock_list = []
    for isin in isin_list:
        r = requests.get(url(isin))
        assert r.status_code == 200, "HTTP Error, {}".format(r.status_code)
        
        html = r.content.decode("utf-8")
        soup = BeautifulSoup(html, 'html.parser')
        stock_dict = get_price_stock(soup)
        stock_dict["ISIN"] = isin
        stock_list.append(stock_dict)
    return(stock_list)

def get_master_data_stock(soup_base):
    assert isinstance(soup_base, type(BeautifulSoup())), "Input is no BeautifulSoup object!"
    metadata = {}
    ### Get name of stock
    stock_name = soup_base.find_all("h1")[0].find_all("span", {"class":"h1"})[0].text
    metadata["Name"] = stock_name
    ### Get metadata from infoboxes: Fondssize, TER
    infoboxes = soup_base.find_all("div", {"class": "infobox"})
    for box in infoboxes:
        value = box.find_all("div", {"class": "val"})[0].text.replace(" ", "").replace("\n", "")
        label = box.find_all("div", {"class": "vallabel"})[0].text.replace(" ", "").replace("\n", "")
        if label == "Fondsgröße":
            assert value[:3] == "EUR", "Fondsgröße not given in EUR!"
            assert value[-4:] == "Mio.", "Fondsgröße not given in Mio EUR!"
            metadata["Fondssize"] = int(float(value[3:-4])*10**6)
        elif label == "Gesamtkostenquote(TER)":
            assert value[-4:] == "p.a.", "TER not given per year!"
            metadata["TER%"] = float(value[:-5].replace(".", "").replace(",", "."))
    ### Get metadata from tables
    tables = soup_base.find_all("table")
    needed_labels = ["Replikationsmethode", "RechtlicheStruktur", "Fondswährung", "Auflagedatum/Handelsbeginn",
                    "Ausschüttung", "Ausschüttungsintervall", "Fondsdomizil", "Fondsstruktur", "Anbieter", 
                    "Depotbank", "Wirtschaftsprüfer"]
    for table in tables:
        bodies = table.find_all("tbody")
        for body in bodies:
            rows = body.find_all("tr")
            for row in rows:
                if len(row.find_all("td")) == 2:
                    label = row.find_all("td")[0].text.replace(" ", "").replace("\n", "")
                    value = row.find_all("td")[1].text.replace(" ", "").replace("\n", "")
                    if label in needed_labels:
                        metadata[label] = value
    return(metadata)

def get_master_data(isin_list):
    stock_list = []
    for isin in isin_list:
        r = requests.get(url(isin))
        assert r.status_code == 200, "HTTP Error, {}".format(r.status_code)
        
        html = r.content.decode("utf-8")
        soup = BeautifulSoup(html, 'html.parser')
        stock_dict = get_master_data_stock(soup)
        stock_dict["ISIN"] = isin
        stock_list.append(stock_dict)
    return(stock_list)

In [3]:
isin = "IE00B1W57M07"
r = requests.get(url(isin))
assert r.status_code == 200, "HTTP Error, {}".format(r.status_code)

html = r.content.decode("utf-8")
soup = BeautifulSoup(html, 'html.parser')

In [4]:
path_all_isin = "/home/chris/Dropbox/Finance/data/ETF_investing.ods"
path_needed_isin = "/home/chris/Dropbox/Finance/data/finanzübersicht.ods"
out_path_master = "/home/chris/Dropbox/Finance/data/master_data_stocks.ods"
out_path_prices = "/home/chris/Dropbox/Finance/stock_prices.ods"

In [5]:
df_in = pd.read_excel(path_all_isin, engine="odf", sheet_name="ETF list")
isin_list_in = list(df_in["ISIN"].drop_duplicates())

In [6]:
len(isin_list_in)

86

In [7]:
dfn = pd.read_excel(path_needed_isin, engine="odf", sheet_name="3.2 Portfolio langfristig Transactions")
isin_list = list(dfn["ISIN"].dropna().drop_duplicates())

In [8]:
len(isin_list)

17

In [9]:
stocks = get_master_data(isin_list_in)
df = pd.DataFrame(stocks)
assert df.count()[0] == len(isin_list_in), "Too less rows!"
df.to_csv(out_path_master, index=False)

In [10]:
prices = get_prices(isin_list)
df_price = pd.DataFrame(prices)
assert df_price.count()[0] == len(isin_list), "Too less rows!"
df_price.to_csv(out_path_prices, index=False)

In [13]:
df_price.head()

,Currency,Price,Datum,ISIN
0,EUR,54.10,30.12.2020,LU0380865021
1,EUR,31.63,30.12.2020,IE00B1W57M07
2,EUR,29.06,30.12.2020,IE00BKM4GZ66
3,EUR,5.93,30.12.2020,IE00BD1F4M44
4,EUR,307.31,30.12.2020,IE00B5BMR087


In [14]:
s = None

In [15]:
type(s)

NoneType

In [16]:
type(None)

NoneType